# Demos of mailchimp API

### Init

Set up credentials and some generic utils.

In [ ]:
from json import load, dumps
from requests import get, post, patch
import pandas as pd
import sys
sys.path.append('../../')

In [ ]:
from api.resources.utils.utils import create_list_of_likes

In [ ]:
base_url = 'https://us3.api.mailchimp.com/3.0/'
MAILCHIMP_CREDENTIALS = '../../api/credentials/mailchimp-key.json'

with open(MAILCHIMP_CREDENTIALS, 'r') as f:
    MAILCHIMP_KEY = load(f)
    
headers = {'content-type': "application/json"}

In [ ]:
test_email = 'steven@stairwaytotravel.com'  # 'steven@stairwaytotravel.com'
page_source = 'bucket_list'  # or, 'about'

And load data for creating custom HTML

In [ ]:
data_dir = '../../api/data/'
file_name = 'wikivoyage_destinations.csv'

### 1. Create user through signup form

Possibly add hidden field for source: `about` or `bucket_list`. Check out this [doc](https://mailchimp.com/help/determine-webpage-signup-location/?&_ga=1.250466117.290251718.1487777734#Before_you_start).

And take into account status `subscribed` versus `non-subscribed` with regards to marketing updates.

In [ ]:
payload = {'email_address': test_email, 
           'status': 'subscribed', 
           'merge_fields': {
               "SIGNUP": page_source
           }}

In [ ]:
r = post(base_url + 'lists/{}/members/'.format(MAILCHIMP_KEY['audience_id']),
         json=payload, headers=headers, auth=('randomstring', MAILCHIMP_KEY['api_key']))

print(r.status_code)

In [ ]:
# r.json()

400 means bad request, makes sense as it's already there. Looks like:

```json
{
    'title': 'Forgotten Email Not Subscribed',
    'status': 400,
    'detail': 'steven@stairwaytotravel.com was permanently deleted and cannot be re-imported. The contact must resubscribe to get back on the list.',
    ...
}
```

Or, if the member already exists:

```json
{
    "title": "Member Exists",
    "status": 400,
    "detail": "steven@stairwaytotravel.com is already a list member. Use PUT to insert or update list members.",
    ...
}
```

If call succeeds, 201 should be returned. However, confusingly, `status` in the succesfull response object now refers to the marketing status of the contact: 

```json
{
    "id": "2a967377119f7d5d3d03d340d8c882f2",
    "email_address": "dude2@gmail.com",
    "status": "subscribed",
    ...
}
```

Anyway, `id` is what we need so return that!

### 2. Check if user exists

And get contact id if true.

Check out example in [docs](https://mailchimp.com/developer/guides/manage-subscribers-with-the-mailchimp-api/)

For the address, urist.mcvankab@freddiesjokes.com, make a GET request to /3.0/lists/9e67587f52/members/62eeb292278cc15f5817cb78f7790b08. However, you can also just use the email address instead of the MD5 hash.

In [ ]:
r = get(base_url + 'lists/{}/members/{}'.format(MAILCHIMP_KEY['audience_id'], test_email),
        auth=('randomstring', MAILCHIMP_KEY['api_key']))

r.json()['status']

In [ ]:
r.status_code == 200

In [ ]:
# r.json()

If the call returns a 404 response, the contact is not subscribed to the list/audience.

```json
{
    "title": "Resource Not Found",
    "status": 404,
    "detail": "The requested resource could not be found.",
    ...
}
```

If the call returns a 200 response, check the status field. 

```json
{
    "id": "a7b7b06c1ad4ec9f0031cd06692301ee",
    "email_address": "steven@stairwaytotravel.com",
    "unique_email_id": "d48f7473bd",
    "status": "transactional",
    ...
}
```

#### Radio buttons and checkboxes

Radio buttons are additional merge fields that can be used to make groups. Checkboxes lead to groups directly. If you haven't liked the `BOOKING` radio and you haven't opted for any checkbox, then your profile looks as follows:

```json
 'merge_fields': {'SIGNUP': '', 'UTIL_HTML': '', 'BOOKING': ''},
 'interests': {'6a6fd52135': False,
  '4fdc15b644': False,
  '7ceb4a1b29': False,
  '621cff108a': False,
  '14af682816': False,
  '5ebcfd6d93': False},
```

If you did enable the `BOOKING` radio button and some interests, this is how it would look like:

```json
'merge_fields': {'SIGNUP': '',
  'UTIL_HTML': '',
  'BOOKING': 'Booking interest'},
 'interests': {'6a6fd52135': False,
  '4fdc15b644': True,
  '7ceb4a1b29': True,
  '621cff108a': False,
  '14af682816': True,
  '5ebcfd6d93': False},
```



### 3. Update contact

For example change someones status from `subscribed` to `transactional`. Check [docs](https://mailchimp.com/developer/guides/manage-subscribers-with-the-mailchimp-api/#Create_or_update_a_contact)

In [ ]:
payload = {
    "status": "subscribed"
}

In [ ]:
r = patch(base_url + 'lists/{}/members/{}'.format(MAILCHIMP_KEY['audience_id'], test_email),
          json=payload, headers=headers, auth=('randomstring', MAILCHIMP_KEY['api_key']))

print(r.status_code)

In [ ]:
# r.json()

If succesfull, returns a 200 with a json of the new member data including the new `status`:

```json
{
    'id': 'a7b7b06c1ad4ec9f0031cd06692301ee',
    'email_address': 'steven@stairwaytotravel.com',
    'status': 'transactional',
    ...
}
```

The patch function can also be used to update other attributes of a user. For example, change one specific merge field called `UTIL_HTML` by setting the payload to:

In [ ]:
payload = {
    "merge_fields": {'UTIL_HTML' : '<ul class="mcnTextContent"> <li>New information. <li>New2 information. <li>New3. </ul>'}
}

This enables us to insert our own HTML to be used email formatting. For example, let's say we want to insert an unordered list based on 5 likes:

In [ ]:
df = pd.read_csv(data_dir + file_name)
likes = df.head()['id'].tolist()

likes

In [ ]:
create_list_of_likes(df, likes)

In [ ]:
payload = {
    "merge_fields": {'UTIL_HTML' : create_list_of_likes(df, likes)}
}

And in the same way, we can also update interests. Interests, however, are accessed through a hash. So you first need to determine the mapping for your categories.

In [ ]:
interest_mapping = {
        'flights': '4fdc15b644',
        'local_transport': '7ceb4a1b29',
        'activities': '621cff108a',
        'accomodation': '14af682816',
        'none': '5ebcfd6d93',
}

But then you can translate any user wishes to a query of the right format:

In [ ]:
interests_user = {
    'flights': False,
    'local_transport': True,
    'activities': True,
    'accomodation': False,
    'none': False,
}

interests_query = {interest_mapping[key]:value for (key,value) in interests_user.items()}


payload = {
    # you can even include other merge fields in the same patch
    "merge_fields": {'BOOKING' : 'No'},
    'interests': interests_query
}

In [ ]:
r = patch(base_url + 'lists/{}/members/{}'.format(MAILCHIMP_KEY['audience_id'], test_email),
          json=payload, headers=headers, auth=('randomstring', MAILCHIMP_KEY['api_key']))

print(r.status_code)

In [ ]:
# r.json()

### 4. Member events

Register someones liked destinations.

Docs [here](https://mailchimp.com/developer/reference/lists/list-members/list-member-events/)

In [ ]:
liked_destinations = [1, 2, 'somePlace', 'test']

payload = {"name": "liked_destinations",
           "properties": {
               "destinations": dumps(liked_destinations),
           }}
headers = {'content-type': "application/json"}

In [ ]:
r = post(base_url + 'lists/{}/members/{}/events'.format(MAILCHIMP_KEY['audience_id'], 
                                                                               test_email),
         json=payload, headers=headers, auth=('randomstring', MAILCHIMP_KEY['api_key']))

print(r.status_code)

Works!

Done.